# Week 4 (Tuesday)- Streaming Data and Text Analysis

**Objectives**: Today we are going to explore streaming data and text analysis. Specifically, we will cover the following:
  
* Streaming Data
* Build code to connect to the Twitter streaming API
* Experiment with that code to generate different streams
* Covert raw streams into structured data for analysis
* Use TextBlob to analyze text

## Streaming Data

Streaming data often embodies all three of the traditional "big data" V's- variety, velocity, and volume. Much of this data is also unstructured. In this context, we are discussing streaming data and not streaming media like video or audio although streaming media is also a component of "big data".

For analytics and streaming data, a typical use case might look like the following example from Amazon Web Services.

<img src="https://raw.githubusercontent.com/azbones/big_data/master/images/Kinesis-Streams_Diagram.png">
(source: https://aws.amazon.com/kinesis/)

A streaming analytics pipeline can be used for many applications including:

* real-time machine learning (recommendations, predictions, etc.)
* real-time analytics
* distributed data generation and collection (JSON activity streams and xAPI)
* Internet of Things data collection

Open source platforms used to handle and analyze streaming data include:

* Apache Kafka for messaging- http://kafka.apache.org/
* Apache Storm for real-time computation- http://storm.apache.org/

Advanced firms like Palantir are expanding beyond the traditional single stream applications like recommendation to have a system built on analyzing and integrating multiple, disparate data streams as can be seen in these product descriptions:

* https://www.palantir.com/solutions/insider-threat/
* https://www.palantir.com/palantir-gotham/

Financial firms are also using Twitter data directly or from aggregators like RavenPack to either inform or in some cases trigger trades.

* http://www.bloomberg.com/bw/articles/2013-04-24/how-many-hft-firms-actually-use-twitter-to-trade
* http://www.ravenpack.com/

A key tool in the example of financial trading is sentiment analysis. Sentiment analysis uses natural language processing and text processing to infer attitudes about the subject of that text. In simplistic financial terms, if the public has positive attitudes about a given firm or its stock, these attitudes will be correlated with price stability or increases in value. Recent examples have even included April Fools Twitter jokes that have seemed to impact stock prices:

* http://blogs.wsj.com/moneybeat/2015/04/01/tesla-stock-moves-on-april-fools-joke/

Real-time analysis of the text data associated with these streams was likely involved in trading decisions like this. More broadly, the use of text analysis and streaming data can be a critical component of any firm's analytics efforts.

Today we are going to get experience with Twitter's streaming API and basic text analysis of that streaming data.

## Twitter Streaming API

Details about the API can be found here:

* https://dev.twitter.com/streaming/overview

Luckily for us, there is also a Python library which makes access to the API fast and easy:

* http://tweepy.readthedocs.org/en/v3.5.0/index.html

**Setting up API Credentials**

To use the Twitter API, you must first register an application with Twitter in order to get the required access credentials. Go to the following website to create an account and register an application so you can get the credentials that are required to run the code below.

* https://dev.twitter.com/

Your apps can be managed at:

* https://apps.twitter.com/

When your application is properly configured, you should be able to access "Keys and Access Tokens" in a page that looks like this:

<img src="https://raw.githubusercontent.com/azbones/big_data/master/images/twitter_apps.png">

**Getting Started**

To get some experience with the API, we will begin by importing a function we built which makes using Tweepy easier.  Insert your Twitter application credentials into the following code block and run it.

In [ ]:
# Import Tweepy functions and include access keys and tokens in global namespace.

from tweet_stream import TwitterAuth, PrintStream, FileStream, get_stream

consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

Next, we need to build a Python connector to the streaming API with your credentials. 

In [ ]:
# Create an OAuth object and make a connector

auth = TwitterAuth(consumer_key, consumer_secret, access_token, access_token_secret)
con = auth.make_connector()

Finally, we need to use the connector and listener to establish a stream. Streaming is fundementally different from other REST APIs given there is an ongoing HTTP connection that is listening for the stream. The following diagram represents this process for Twitter:

<img src="https://raw.githubusercontent.com/azbones/big_data/master/images/twitter_streaming-intro-2_1.png">
(source: https://dev.twitter.com/streaming/overview)

We have defined a function that is a listener to retrieve the data as it is streamed to your connection.  This listener was built using the <code>PrintStream()</code> class which defines that the results of the stream should be printed to the console.

**Print to Console**

Once the stream is set up, we are using the <code>filter</code> function to pass an array of search values in the <code>track</code> parameter which the Twitter API uses to pass to the connection. Try entering some search terms in the <code>filter</code> function in the next code block to begin printing output to your notebook.

**Please Note:** The data you receive from Twitter may contain inappropriate material, so  use caution if you are sensitive to any particular content.

Given this is a streaming connection, the output from the API will continue until you cancel it.  Depending on the volume of your filter terms, the API make take some time before it starts sending you data. To cancel it, use the "interrupt kernel" button which is a black square in the Jupyter UI.

In [ ]:
# Set up listener and start stream with defined search terms.

listener = PrintStream()
stream = get_stream(con, listener)
stream.filter(track=[''])

While you may have stopped the console printing, the stream is most likely still active which you can check by calling the stream's <code>running</code> attribute. Do that in the next code block.

In [ ]:
# Check if the stream is still running

stream.running

To stop the connection, use the <code>disconnect()</code> function on the stream and then check if it is still running.

In [ ]:
# Disconnect the stream

stream

In [ ]:
# Check to see if the stream is still active.

stream

**Print to File**

Next, instead of printing the output of the API to the console, let's write it to a file. To stop the listener, you again have to interrupt the kernel and disconnect the listener. This listener is using a the function we built called <code>FileStream</code> that appends the results from the listener to a filename you specify. Complete the following code then check to see that the file is being created in your file system. Wait until the file reaches a reasonable size and then stop and disconnect the stream.

In [ ]:
# Set up listener and start stream with defined search terms.

listener = FileStream(filepath='filename')
stream = get_stream(con, listener)
stream.filter(track=[''])

**Read Data From File**

Finally, let's open that file to read the JSON contents into a file as a list of Python dictionaries. We built a small function to make this easier called <code>tweets_list</code>.

Each JSON document contains a range of different values most of which can be referenced here:

* https://dev.twitter.com/overview/api/tweets


In [ ]:
import json

def tweets_list(filename):
    """
    Read lines from filepath and file into a list of dictionaries.
    
    Parameters
    ----------
    filename: str
    """
    tweets = []
    f = open(filename, 'r')
    for line in f:
        try:
            tweet = json.loads(line)
            tweets.append(tweet)
        except:
            continue
    return tweets     

Run the code above to read the <code>tweets_list</code> function into the namespace.  Then, use the function to read your tweets file into a list of dictionaries and assign it to a variable so you can explore it more.

In [ ]:
# Read your tweet file with tweets_list


<code>tweets_list</code> returns a list of the JSON documents in dictionary format from the file you specified. Use Python's list index to explore different tweets.  Within individual tweets, use Python's dictionary key-value access method to explore specific data. The Pretty Print module is useful in situations like this as it will better format print output.

* https://docs.python.org/2/library/pprint.html

In [ ]:
# Explore tweet data structure


Next, use pandas to read the tweets into a dataframe in the next codeblock. Each document may have a different number of first level keys, so pandas will use the max number of keys to define columns and then apply NaN (or null values) in those cases where the keys are not present.

The general syntax to accomplish this is:

```
dataframe = pandas.DataFrame(list of Python dictionaries)
```

In [ ]:
# Create dataframe from list of dicts


Next, examine the dataframe to get a sense for what elements are included in it. Be aware that the Twitter JSON was nested, so there will be elements in the dataframe that are of type <code>dict</code>. Use slicing and indexing to evaluate individual elements.

One interesting example of nesting is the <code>retweeted_status</code> which is a complete representation of the original tweet that was retweeted.

In [ ]:
# Examine the dataframe


## Natural Language Processing (NLP) and Text Analysis

The analysis of unstructured text data like that in tweets is the domain of natural language processing (NLP). NLP is a significant area of study for computer scientists both in academia and industry. These efforts include dedicated research groups like:

* **Stanford NLP Group:** http://nlp.stanford.edu/
* **Google NLP:** http://research.google.com/pubs/NaturalLanguageProcessing.html
* **Microsoft NLP:** http://research.microsoft.com/en-us/groups/nlp/

In this lab, we are going to provide a very basic introduction to text analysis using the TextBlog Python Library:

* https://textblob.readthedocs.org/en/dev/

TextBlob offers a simple interface for experimenting with NLP. It relies on the more robust Python Natural Language Toolkit or NLTK.

* http://www.nltk.org/

In addition to the code for conducting NLP analyses, you typically also need a corpus or dataset which provides the equivilent of metadata for text data.  For example, to classify a word as a "noun" requires a dataset with that word and the word's part of speech which can be looked up based on the word itself and potentially its context within the text. So, for our purposes, we also need to install the NLTK corpora.

* http://www.nltk.org/data.html

**Read Some Text into Memory**

To experiment with TextBlob, we will start with opening the "alice.txt" file from the "datasets" folder in this directory. We will then assign the contents of the file to a variable in our namespace. Run the code in the next text block to do that.


> A quick aside: Python 2.x can be challenging to work with text data given Python type <code>str</code> doesn't explicitly support unicode.  This blog post is a good summary of the issue- http://www.azavea.com/blogs/labs/2014/03/solving-unicode-problems-in-python-2-7/. The code below uses the codecs module to create a <code>unicode</code> type object.

In [ ]:
# Create a file object and read its contents into a variable as unicode.

import codecs
with codecs.open('datasets/alice.txt','r',encoding='utf8') as f:
    text = f.read()

In [ ]:
# Check the type of the text object here


**Create a TextBlob Object and Derive Basic Statistics**

Now, we will import TextBlob and create an object from our text variable that we can use to try several different NLP methods.

In [ ]:
from textblob import TextBlob

blob = TextBlob(text)

TextBlob has an intuitive API which makes basic text analysis quite easy. It can also be used to create custom text classifiers. 

* **The Basics:** https://textblob.readthedocs.org/en/dev/quickstart.html
* **Building a Text Classifier:** https://textblob.readthedocs.org/en/dev/classifiers.html

Try some of the basics below.

In [ ]:
# Create a word count

blob.word_counts

In [ ]:
# Print out noun phrases

blob.noun_phrases

In [ ]:
# Count noun phrases

blob.np_counts

In [ ]:
# Derive https://textblob.readthedocs.org/en/dev/api_reference.html#textblob.blob.TextBlob.sentiment

blob.sentiment